In [1]:
from joblib import dump, load
import numpy as np 
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string, re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from flask import Flask, request, jsonify
from flask_cors import CORS

In [2]:
app = Flask(__name__)
CORS(app)

In [3]:
clf_rnd = load('rnd.joblib') 
bow= load('bow.joblib')

In [4]:
def clean_tweet(tweet):
    lemmatizer = WordNetLemmatizer() # lemmatizer return word in it's base noun (root)
    twt_tokenizer= TweetTokenizer(strip_handles=True) # tokenizer convert string into token
    tokens = twt_tokenizer.tokenize(tweet) 
    stops = stopwords.words('english')+ list(string.punctuation) # list of stopwords and punctuation
    no_stops_sentence = []
    for w in tokens: # removing stop words
        if w not in stops:
            no_stops_sentence.append(w)
    no_hashtag_sentence = []
    for w in no_stops_sentence: # removing # from words using regex
        no_hashtag_sentence.append(re.sub('#','',w))
    no_url_sentence=[]
    for w in no_hashtag_sentence: # removing www from urls using regex
        no_url_sentence.append(re.sub('www\S','',w))
    no_http_sentence=[]
    for w in no_url_sentence: # removing http from urls using regex
        no_http_sentence.append(re.sub('http','',w))
    lemma_sentence=[]
    for w in no_http_sentence: # lemmatizing all words to it's root
        lemma_sentence.append(lemmatizer.lemmatize(w))
    final_tokens=[]
    for w in lemma_sentence: # removing no length words
        if len(w)>1:
            final_tokens.append(w)
    final_sentence=""
    for w in final_tokens:
        final_sentence=final_sentence+' '+w
    return final_sentence


        
    

In [5]:
@app.route('/predict/comment',methods=['POST'])
def predict_api():
    json_data = request.json
    cleaned_text=clean_tweet(json_data['comment'])
    transformed_text=bow.transform([cleaned_text])
    prediction_proba=clf_rnd.predict_proba(transformed_text)
    if(prediction_proba[0][0]>0.65):
        result=0
    else:
        result=1

    return jsonify({'result':result})  #will return the json


In [ ]:
if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
